In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,random_split
import torch.optim as optim
from torchvision.transforms import transforms
from PIL import Image
import scipy
import os


from utils import OxfordDataset

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
])

In [4]:
dataset = OxfordDataset("flower_data", transform=transform)


In [5]:
dataset[0]

(tensor([[[-0.4568, -0.4739, -0.4739,  ...,  0.1426,  0.1426,  0.0912],
          [-0.4397, -0.4739, -0.4739,  ...,  0.2282,  0.2453,  0.2624],
          [-0.3883, -0.4226, -0.4568,  ...,  0.3652,  0.3481,  0.3309],
          ...,
          [-1.2959, -1.2959, -1.3130,  ..., -1.3815, -1.4158, -1.4500],
          [-1.3815, -1.3815, -1.3644,  ..., -1.3302, -1.2959, -1.3130],
          [-1.4500, -1.4500, -1.4500,  ..., -1.1075, -1.0733, -1.1247]],
 
         [[ 0.4853,  0.5203,  0.5728,  ...,  0.5203,  0.5028,  0.4328],
          [ 0.5378,  0.5553,  0.5903,  ...,  0.6954,  0.6604,  0.6078],
          [ 0.5903,  0.5728,  0.5903,  ...,  0.8529,  0.8179,  0.7654],
          ...,
          [-1.0028, -1.0028, -1.0028,  ..., -1.1954, -1.2129, -1.2479],
          [-1.0903, -1.0903, -1.1253,  ..., -1.0553, -1.0553, -1.1429],
          [-1.1779, -1.1779, -1.1954,  ..., -0.8277, -0.8452, -0.9678]],
 
         [[-1.4733, -1.5256, -1.5953,  ..., -0.3578, -0.3578, -0.4101],
          [-1.4559, -1.5430,

In [6]:
train_split = int(0.70 * len(dataset))
validation_split = int(0.15 * len(dataset))
test_split = len(dataset) - train_split - validation_split

train_dataset, validation_split, test_split = random_split(dataset, [train_split, validation_split, test_split])



In [7]:
train_loader = DataLoader(train_dataset,batch_size=32, shuffle=True)
validation_loader = DataLoader(validation_split,batch_size=32, shuffle=False)
test_loader = DataLoader(test_split, batch_size=32, shuffle=False)


In [8]:
for image,label in train_loader:
    print(label)
    print(label[0].item())
    print(image.shape)
    break



tensor([ 53,  59,  50,  50,  59,  92,  87,  87,  81,  81,  65,  74,  87,   2,
         86,  88,  49,  71,  50,  55,  87,  63,  72,  73,  50,  58,  85,  18,
         45, 100,  64,  38], dtype=torch.uint8)
53
torch.Size([32, 3, 224, 224])


In [9]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32,kernel_size=3,padding=1,stride=2)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2,stride=2)

        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64,kernel_size=3,stride=2,padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128,kernel_size=3,stride=1,padding=1)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv4 = nn.Conv2d(in_channels=128, out_channels=512,kernel_size=3,stride=1,padding=1)
        self.relu4 = nn.ReLU()
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(512*3*3,512)
        self.relu4 = nn.ReLU()
        self.dropout = nn.Dropout(0.7)
        self.fc2 = nn.Linear(512,102)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)  
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.pool3(x)

        x = self.conv4(x)
        x = self.relu4(x)
        x = self.pool4(x)
        
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu4(x)
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x



In [10]:
model = SimpleCNN().to(device)

optimizer = optim.Adam(model.parameters(),lr=0.0005, weight_decay=0.0005)
loss_function = nn.CrossEntropyLoss()


In [11]:
def train_epoch(model, train_loader, optimizer, loss_function):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for batch_idx, (data, labels) in enumerate(train_loader):
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_function(output, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = output.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        if (batch_idx + 1) % 10 == 0:  # Changed condition
            avg_loss = running_loss / 10  # Now correct
            acc = 100. * correct / total
            print(f' [{(batch_idx + 1) * len(data)}/{len(train_loader.dataset)}]'
                  f' Loss: {avg_loss:.3f} | Accuracy: {acc:.1f} %')
            running_loss = 0.0  # Only reset loss, keep accuracy cumulative

In [12]:
def validation(model,validation_loader,device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for idx, (data, targets) in enumerate(validation_loader):
            data, targets = data.to(device), targets.to(device)
            output = model(data)
            _,predicted = output.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    
    return 100. * correct/total


In [ ]:
num_epochs = 10

for idx in range(num_epochs):
    print(f"Epoch : {idx}")
    train_epoch(model,train_loader,optimizer,loss_function)
    validation_acc = validation(model, validation_loader, device)
    print(f"Validation Accuracy : {validation_acc}")



Epoch : 0
 [320/5732] Loss: 4.631 | Accuracy: 0.3 %
 [640/5732] Loss: 4.586 | Accuracy: 1.7 %
 [960/5732] Loss: 4.591 | Accuracy: 2.1 %
 [1280/5732] Loss: 4.589 | Accuracy: 2.1 %
 [1600/5732] Loss: 4.521 | Accuracy: 2.6 %
 [1920/5732] Loss: 4.521 | Accuracy: 2.7 %
 [2240/5732] Loss: 4.477 | Accuracy: 2.7 %
 [2560/5732] Loss: 4.399 | Accuracy: 2.6 %
 [2880/5732] Loss: 4.333 | Accuracy: 2.7 %
 [3200/5732] Loss: 4.325 | Accuracy: 2.8 %
 [3520/5732] Loss: 4.355 | Accuracy: 3.0 %
 [3840/5732] Loss: 4.267 | Accuracy: 3.3 %
 [4160/5732] Loss: 4.186 | Accuracy: 3.6 %
 [4480/5732] Loss: 4.191 | Accuracy: 3.7 %
 [4800/5732] Loss: 4.168 | Accuracy: 3.9 %
 [5120/5732] Loss: 4.207 | Accuracy: 4.0 %
 [5440/5732] Loss: 4.151 | Accuracy: 4.0 %
 [720/5732] Loss: 4.075 | Accuracy: 4.1 %
Validation Accuracy : 7.49185667752443
Epoch : 1
 [320/5732] Loss: 4.057 | Accuracy: 7.5 %
 [640/5732] Loss: 4.046 | Accuracy: 7.7 %
 [960/5732] Loss: 3.956 | Accuracy: 7.5 %
 [1280/5732] Loss: 3.970 | Accuracy: 7.5 %
 [